# 实现一个带有可训练权重的自注意力

* 与简单自注意力的区别：模型训练期间会更新权重矩阵，使得模型能够学习“良好”的上下文向量。

## 步骤

1. 初始查询 $W_q$ ，键 $W_k$，值 $W_v$ 权重矩阵。
	* 然后 `input` 分别 `矩阵乘法 @` 三个权重矩阵，得到 `queries`，`keys`，`values`三个向量。
2. 计算注意力权重：通过查询 queries 和键 keys 向量
	* 不是简单自注意力直接计算输入向量之间的点积，而是通过：
		* `attn_scores = queries @ keys.T`
	* 缩放点积归一化
		* `attn_weights = torch.softmax(attn_scores / keys.shape[-1] ** 0.5, dim = -1)`
3. 计算上下文向量：通过注意力权重和值 values 向量
	* `context_vec = attn_weights @ values`

## exp - 以输入变量 x(2) 计算上下文向量 z(2)

### 定义输入

In [1]:
# 定义输入向量：

import torch
inputs = torch.tensor(
    [[0.43, 0.15, 0.89], # Your     (x^1)
    [0.55, 0.87, 0.66], # journey  (x^2)
    [0.57, 0.85, 0.64], # starts   (x^3)
    [0.22, 0.58, 0.33], # with     (x^4)
    [0.77, 0.25, 0.10], # one      (x^5)
    [0.05, 0.80, 0.55]] # step     (x^6)   
)

# 类GPT模型d_in = d_out，
# 为了更好展示计算过程，我们选择了不同的输入和输出
x_2 = inputs[1]
d_in = inputs.shape[1]
d_out = 2

print("x_2:\n", x_2)

x_2:
 tensor([0.5500, 0.8700, 0.6600])


### 引入三个权重矩阵 Wq，Wk 和 Wv

In [2]:
# 定义三个权重矩阵Wq，Wk和Wv

torch.manual_seed(123)

# requires_grad 指示这个参数是否需要计算梯度
# 设置为 False 是为了示范时输出结果更清晰
# 但如果我们要将这些权重矩阵用于模型训练，
# 我们会将 requires_grad 设置为 True，
# 以便在模型训练期间更新这些矩阵
W_query = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)
W_key   = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)
W_value = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)

print("W_query:\n", W_query)

W_query:
 Parameter containing:
tensor([[0.2961, 0.5166],
        [0.2517, 0.6886],
        [0.0740, 0.8665]])


### 计算查询，键和值向量

In [3]:
# 计算查询、键和值向量：

query_2 = x_2 @ W_query
key_2 = x_2 @ W_key 
value_2 = x_2 @ W_value

print("query_2:\n", query_2)

query_2:
 tensor([0.4306, 1.4551])


In [4]:
# 通过矩阵乘法获得所有键和值向量：

querys = inputs @ W_query # 
keys = inputs @ W_key # 6 * 3 @ 3 * 2
values = inputs @ W_value

print("querys:\n", querys)
print("keys.shape:\n", keys.shape)

querys:
 tensor([[0.2309, 1.0966],
        [0.4306, 1.4551],
        [0.4300, 1.4343],
        [0.2355, 0.7990],
        [0.2983, 0.6565],
        [0.2568, 1.0533]])
keys.shape:
 torch.Size([6, 2])


### 计算注意力得分

* 公式：$$w_{2i} = ((inputs @ W\_query)[i]).dot((inputs @ W\_key)[i])$$

In [5]:
keys_2 = keys[1] # 1 * 2

attn_score_22 = query_2.dot(keys_2)
print(attn_score_22)

# x_2的所有的注意力得分：

attn_score_2 = query_2 @ keys.T
print(attn_score_2)

tensor(1.8524)
tensor([1.2705, 1.8524, 1.8111, 1.0795, 0.5577, 1.5440])


### 计算注意力权重

In [6]:
# 缩放点积注意力：除以键的嵌入维度的平方根来缩放注意力得分
d_k = keys.shape[-1]

attn_weights_2 = torch.softmax(attn_score_2 / d_k ** 0.5, dim=0)
print(attn_weights_2)
print(attn_weights_2.sum())

tensor([0.1500, 0.2264, 0.2199, 0.1311, 0.0906, 0.1820])
tensor(1.)


#### 缩放注意力

> 对于类似 GPT 的大语言模型，其维度通常超过千，较大的点积可能会因为应用了 softmax 函数而在反向传播过程中产生非常小的梯度，减缓学习速度或导致训练停滞

In [7]:
context_vec_2 = attn_weights_2 @ values
print(context_vec_2)

tensor([0.3061, 0.8210])


## 代码 - SelfAttention_v1

In [8]:
import torch.nn as nn
class SelfAttention_v1(nn.Module):
    def __init__(self, d_in, d_out):
        super().__init__()
        self.d_out = d_out
        # 默认 requires_grad = True 需要计算梯度以便模型训练期间更新这些矩阵
        self.W_query = nn.Parameter(torch.rand(d_in, d_out))
        self.W_key = nn.Parameter(torch.rand(d_in, d_out))
        self.W_value = nn.Parameter(torch.rand(d_in, d_out))
        
    def forward(self, x):
        keys = x @ self.W_key
        queries = x @ self.W_query
        values = x @ self.W_value
        attn_scores = queries @ keys.T
        attn_weights = torch.softmax(
            attn_scores / keys.shape[-1] ** 0.5, dim = -1)
        context_vec = attn_weights @ values
        return context_vec

### 测试

In [9]:
# 测试使用该v1：

torch.manual_seed(123)
sa_v1 = SelfAttention_v1(d_in, d_out)
print(inputs)
print(sa_v1(inputs))

tensor([[0.4300, 0.1500, 0.8900],
        [0.5500, 0.8700, 0.6600],
        [0.5700, 0.8500, 0.6400],
        [0.2200, 0.5800, 0.3300],
        [0.7700, 0.2500, 0.1000],
        [0.0500, 0.8000, 0.5500]])
tensor([[0.2996, 0.8053],
        [0.3061, 0.8210],
        [0.3058, 0.8203],
        [0.2948, 0.7939],
        [0.2927, 0.7891],
        [0.2990, 0.8040]], grad_fn=<MmBackward0>)


## 代码 - SelfAttention_v2

In [10]:
# 使用Pytorch的Linear层的自注意力类

import torch.nn as nn
class SelfAttention_v2(nn.Module):
    def __init__(self, d_in, d_out, qkv_bias=False):
        super().__init__()
        self.d_out = d_out
        # v1: self.W_query = nn.Parameter(torch.rand(d_in, d_out))
        # nn.Linear 采用了比 nn.Parameter 更为复杂的权重初始化方案
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        
    def forward(self, x):
        # v1: x @ self.W_key
        keys = self.W_key(x)
        queries = self.W_query(x)
        values = self.W_value(x)
        attn_scores = queries @ keys.T
        attn_weights = torch.softmax(
            attn_scores / keys.shape[-1] ** 0.5, dim = -1)
        context_vec = attn_weights @ values
        return context_vec

### 测试

In [11]:
# 测试v2：

import torch

torch.manual_seed(789)
sa_v2 = SelfAttention_v2(d_in, d_out)
print(inputs)
print(sa_v2(inputs))

tensor([[0.4300, 0.1500, 0.8900],
        [0.5500, 0.8700, 0.6600],
        [0.5700, 0.8500, 0.6400],
        [0.2200, 0.5800, 0.3300],
        [0.7700, 0.2500, 0.1000],
        [0.0500, 0.8000, 0.5500]])
tensor([[-0.0739,  0.0713],
        [-0.0748,  0.0703],
        [-0.0749,  0.0702],
        [-0.0760,  0.0685],
        [-0.0763,  0.0679],
        [-0.0754,  0.0693]], grad_fn=<MmBackward0>)


## 练习3.1比较 SelfAttention_v1 和 SelfAttention_v2

### 题目

请注意，SelfAttention_v2 中的 nn.Linear 使用了与 SelfAttention_v1 中的 nn.Parameter(torch.rand(d_in, d_out)) 不同的权重初始化方案，这导致两种机制产生不同的结果。为了检查 SelfAttention_v1 和 SelfAttention_v2 的实现在其他方面是否相似，我们可以将SelfAttention_v2对象的权重矩阵转移到SelfAttention_v1对象上，以便两个对象生成相同的结果。

你的任务是正确地将SelfAttention_v2实例的权重分配给SelfAttention_v1实例。为此，你需要理解两种版本中权重之间的关系。（提示：nn.Linear以转置形式存储权重矩阵。）完成权重分配后，你应该会发现两个实例生成相同的输出。

### 解决

In [12]:
import torch.nn as nn
class SelfAttention_v1_v2(nn.Module):
    def __init__(self, d_in, d_out):
        super().__init__()
        self.d_out = d_out
        # 用v2生成的矩阵
        self.W_query = sa_v2.W_query.weight.data.T
        self.W_key = sa_v2.W_key.weight.data.T
        self.W_value = sa_v2.W_value.weight.data.T
        
    def forward(self, x):
        keys = x @ self.W_key
        queries = x @ self.W_query
        values = x @ self.W_value
        attn_scores = queries @ keys.T
        attn_weights = torch.softmax(
            attn_scores / keys.shape[-1] ** 0.5, dim = -1)
        context_vec = attn_weights @ values
        return context_vec
    
    
torch.manual_seed(123)
sa_v2 = SelfAttention_v1_v2(d_in, d_out)
print(inputs)
print(sa_v2(inputs))

tensor([[0.4300, 0.1500, 0.8900],
        [0.5500, 0.8700, 0.6600],
        [0.5700, 0.8500, 0.6400],
        [0.2200, 0.5800, 0.3300],
        [0.7700, 0.2500, 0.1000],
        [0.0500, 0.8000, 0.5500]])
tensor([[-0.0739,  0.0713],
        [-0.0748,  0.0703],
        [-0.0749,  0.0702],
        [-0.0760,  0.0685],
        [-0.0763,  0.0679],
        [-0.0754,  0.0693]])
